In [2]:
import pandas as pd
import nltk
import numpy as np
import os
import pprint

In [3]:
!ls

CLDR.ipynb		  embeddings.py        slurm-1085587.out  Thesis
config_cpu.sh		  nltk_data	       slurm-1090340.out  virtual_env
config_gpu.sh		  Preprocessing.ipynb  slurm-1104361.out
doc2vec_embeddings.ipynb  __pycache__	       start_env.sh


In [4]:
nltk.download('wordnet')    

if "thesis_code" not in os.listdir("Thesis"):
    !git clone https://github.com/marc-lenz/thesis_code.git

%cd Thesis
from thesis_code.Preprocessor import Preprocessor

[nltk_data] Downloading package wordnet to
[nltk_data]     /home2/s4231317/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home2/s4231317/nltk_data...


/home2/s4231317/Thesis


[nltk_data]   Package punkt is already up-to-date!


In [5]:
languages = ["en", "ro", "es", "fr", "de", "nl"]

df = pd.read_csv("Data/data_merged.csv")

In [6]:
df.head()

,number,body_en,body_ro,body_es,body_fr,body_de,body_nl
0,31969R2264,Regulation (EEC) No 2264/69 of the Commissio...,REGULAMENTUL COMISIEI (CEE) nr. 2264/69 din...,"Reglamento (CEE) nº 2264/69 de la Comisión, ...","Règlement (CEE) n° 2264/69 de la Commission,...",Verordnung (EWG) Nr. 2264/69 der Kommission ...,Verordening (EEG) nr. 2264/69 van de Commiss...
1,31969L0466,Council Directive 69/466/EEC of 8 December 1...,DIRECTIVA 69/466/CEE A CONSILIULUI din 8 de...,"Directiva 69/466/CEE del Consejo, de 8 de di...","Directive 69/466/CEE du Conseil, du 8 décemb...",Richtlinie 69/466/EWG des Rates vom 8. Dezem...,Richtlijn 69/466/EEG van de Raad van 8 decem...
2,31969R2602,Regulation (EEC) No 2602/69 of the Council o...,REGULAMENTUL (CEE) nr. 2602/69 AL CONSILIUL...,"Reglamento (CEE) nº 2602/69 del Consejo, de ...","Règlement (CEE) n° 2602/69 du Conseil, du 18...",Verordnung (EWG) Nr. 2602/69 des Rates vom 1...,Verordening (EEG) nr. 2602/69 van de Raad va...
3,31969L0464,Council Directive 69/464/EEC of 8 December 1...,DIRECTIVA 69/464/CEE A CONSILIULUI din 8 de...,"Directiva 69/464/CEE del Consejo, de 8 de di...","Directive 69/464/CEE du Conseil, du 8 décemb...",Richtlinie 69/464/EWG des Rates vom 8. Dezem...,Richtlijn 68/464/EEG van de Raad van 8 decem...
4,31969L0063,Council Directive 69/63/EEC of 18 February 1...,DIRECTIVA CONSILIULUI din 18 februarie 1969...,"Directiva 69/63/CEE del Consejo, de 18 de fe...","Directive 69/63/CEE du Conseil, du 18 févrie...",Richtlinie 69/63/EWG des Rates vom 18. Febru...,Richtlijn 69/63/EEG van de Raad van 18 febru...


In [7]:
from tqdm import tqdm 

if "tok_data.pickle" not in os.listdir("Data"):
    preprocessor_en = Preprocessor(language="en")
    preprocessor_fr = Preprocessor(language="fr")
    preprocessor_other = Preprocessor(language="general")

    processed_data = []

    for lang in languages:
        if lang == "en":
            preprocessor = preprocessor_en
        if lang == "fr":
            preprocessor = preprocessor_fr
        else:
            preprocessor = preprocessor_other
        aux = [preprocessor.preprocess(doc) for doc in tqdm(df[f"body_{lang}"], desc=lang)]
        processed_data.append(aux)
    
    data_dict = {}
    for lang, dat in zip(languages, processed_data):
      key = f"body_pre_{lang}"
      data_dict[key] = dat

    df_pre = pd.DataFrame(data_dict)
    df_pre.to_pickle("Data/tok_data.pickle")
else:
    print("Data was already tokenized - loading data")
    df_pre = pd.read_pickle("Data/tok_data.pickle")
    
train_df_pre = df_pre[:4000]
test_df_pre = df_pre[4000:5000]
val_df_pre = df_pre[5000:]

# train_df_pre = df_pre[:10]
# test_df_pre = df_pre[10:15]
# val_df_pre = df_pre[15:20]

# dimension = 2000

Data was already tokenized - loading data


In [8]:
print(np.shape(train_df_pre))
print(np.shape(test_df_pre))
print(np.shape(val_df_pre))

dimensions = [250, 500, 750, 768, 1000, 1250, 1500, 1750, 2000]

(4000, 6)
(1000, 6)
(1487, 6)


In [9]:
# from gensim.test.utils import common_texts
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# import time

# matrices = dict()
# for t in tqdm(languages):
#   key = "body_pre_{}".format(t)
#   #create tagged docs first
#   documents = []
#   for ind in train_df_pre.index:
#     doc = train_df_pre[key][ind]
#     tagged_doc = TaggedDocument(doc, [ind])
#     documents.append(tagged_doc)

#   #Train Doc2Vec Model
#   model = Doc2Vec(documents, vector_size=dimension, window=4, min_count=5, workers=120, epochs=100, dm=0)
#   training_docs = [model[i] for i in train_df_pre.index]

#   ##### ???????????????????????
#   # validation_docs = [model[i] for i in val_df.index]
#   # test_docs = [model[i] for i in test_df_pre.index]
#   ##### ???????????????????????
#   #set matrices
#   matrices["{}_train_vecs".format(t)] = training_docs
#   test = []
#   for idx in test_df_pre.index:
#     p = model.infer_vector(test_df_pre[key][idx])
#     test.append(p)
#   matrices["{}_test_vecs".format(t)] = test

#   val = []
#   for idx in val_df_pre.index:
#     p = model.infer_vector(val_df_pre[key][idx])
#     val.append(p)
#   matrices["{}_val_vecs".format(t)] = val
#   # matrices[dimension]["{}_val_vecs".format(t)] = np.asarray(validation_docs)
#   # matrices["{}_test_vecs".format(t)] = np.asarray(test_docs)

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import time

%cd Embeddings
!mkdir doc2vec

for dimension in dimensions:
    matrices = dict()
    
    if f"emb_doc2vec_train_{dimension}.pickle" in os.listdir("doc2vec"):
        print(f"{dimension} dimension already present")
        continue
    
    for t in tqdm(languages, desc=f"{dimension}"):
      key = "body_pre_{}".format(t)
      #create tagged docs first
      documents = []
      for ind in train_df_pre.index:
        doc = train_df_pre[key][ind]
        tagged_doc = TaggedDocument(doc, [ind])
        documents.append(tagged_doc)

      #Train Doc2Vec Model
      model = Doc2Vec(documents, vector_size=dimension, window=4, min_count=5, workers=50, epochs=100, dm=0)
      training_docs = [model[i] for i in train_df_pre.index]

      ##### ???????????????????????
      # validation_docs = [model[i] for i in val_df.index]
      # test_docs = [model[i] for i in test_df_pre.index]
      ##### ???????????????????????
      #set matrices
      matrices["{}_train_vecs".format(t)] = training_docs
      test = []
      for idx in test_df_pre.index:
        p = model.infer_vector(test_df_pre[key][idx])
        test.append(p)
      matrices["{}_test_vecs".format(t)] = test

      val = []
      for idx in val_df_pre.index:
        p = model.infer_vector(val_df_pre[key][idx])
        val.append(p)
      matrices["{}_val_vecs".format(t)] = val
      # matrices[dimension]["{}_val_vecs".format(t)] = np.asarray(validation_docs)
      # matrices["{}_test_vecs".format(t)] = np.asarray(test_docs)
    aux_dict = {}
    for lang in languages:
      aux_dict[f"{lang}_train_vecs"] = matrices[f"{lang}_train_vecs"]

    doc2vec_emb_df_train = pd.DataFrame(aux_dict)
    doc2vec_emb_df_train.to_pickle(f"doc2vec/emb_doc2vec_train_{dimension}.pickle")

    aux_dict = {}
    for lang in languages:
      aux_dict[f"{lang}_test_vecs"] = matrices[f"{lang}_test_vecs"]

    doc2vec_emb_df_test = pd.DataFrame(aux_dict)
    doc2vec_emb_df_test.to_pickle(f"doc2vec/emb_doc2vec_test_{dimension}.pickle")

    aux_dict = {}
    for lang in languages:
      aux_dict[f"{lang}_val_vecs"] = matrices[f"{lang}_val_vecs"]

    doc2vec_emb_df_val = pd.DataFrame(aux_dict)
    doc2vec_emb_df_val.to_pickle(f"doc2vec/emb_doc2vec_val_{dimension}.pickle")

%cd ..

/home2/s4231317/Thesis/Embeddings
mkdir: cannot create directory ‘doc2vec’: File exists
250 dimension already present
500 dimension already present
750 dimension already present
768 dimension already present
1000 dimension already present
1250 dimension already present
1500 dimension already present


1750:  17%|█▋        | 1/6 [47:03<3:55:18, 2823.70s/it]

In [ ]:
len(matrices["es_val_vecs"])

In [10]:
%cd Embeddings
!mkdir doc2vec

aux_dict = {}
for lang in languages:
  aux_dict[f"{lang}_train_vecs"] = matrices[f"{lang}_train_vecs"]

doc2vec_emb_df_train = pd.DataFrame(aux_dict)
doc2vec_emb_df_train.to_pickle(f"doc2vec/emb_doc2vec_train_{dimension}.pickle")

aux_dict = {}
for lang in languages:
  aux_dict[f"{lang}_test_vecs"] = matrices[f"{lang}_test_vecs"]

doc2vec_emb_df_test = pd.DataFrame(aux_dict)
doc2vec_emb_df_test.to_pickle(f"doc2vec/emb_doc2vec_test_{dimension}.pickle")

aux_dict = {}
for lang in languages:
  aux_dict[f"{lang}_val_vecs"] = matrices[f"{lang}_val_vecs"]

doc2vec_emb_df_val = pd.DataFrame(aux_dict)
doc2vec_emb_df_val.to_pickle(f"doc2vec/emb_doc2vec_val_{dimension}.pickle")

%cd ..

/home2/s4231317/Thesis/Embeddings
mkdir: cannot create directory ‘doc2vec’: File exists
/home2/s4231317/Thesis
